In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
year = 2014

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [4]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['historical'], table_id=['day'], variable_id=['hus','va'], member_id=['r1i1p1f1'])

In [6]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [7]:
dataset_list = list(dset_dict.keys())

In [8]:
dset = dset_dict[dataset_list[0]]
dset = dset.sel(member_id='r1i1p1f1',time=slice(str(year)+"-01-01", str(year)+"-12-31"))

In [9]:
vas = dset.va
hus = dset.hus
plev = dset.plev
lat_ = hus.lat
lon_ = hus.lon

In [10]:
ivt = compute_ivt(hus,vas,plev)

In [11]:
ivt_ns = ivt.copy()
ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward

In [12]:
# pre-computed 98th percentile IVT
q98 = xr.open_dataset('q98.nc')
q98 = q98.rename({'__xarray_dataarray_variable__':'ivt'})

In [13]:
excess = ivt_ns_pos-q98
ar_points = xr.where(excess>0,1,0,keep_attrs=True)

In [18]:
out_ar = ar_points.copy()
out_ar.ivt.values[:] = out_ar.ivt.values[:]*0.0

res_lon = abs(lon_.values[1]-lon_.values[0])
res_lat = abs(lat_.values[1]-lat_.values[0])

def to_nc(indata, lat_test, lon_test, tts, fln):
    nc = Dataset(fln, 'w')
    lat_dim = nc.createDimension('lat', indata.shape[1])
    lon_dim = nc.createDimension('lon', indata.shape[2])
    t_dim = nc.createDimension('time', len(tts))
    lat_var = nc.createVariable('lat', np.float64, ('lat'))
    lat_var[:] = lat_test
    lon_var = nc.createVariable('lon', np.float64, ('lon'))
    lon_var[:] = lon_test
    tnd = nc.createVariable('ivt', np.float64, ('time','lat','lon'))
    tnd[:,:,:] = indata
    times = nc.createVariable('time', np.float64, ('time'))
    times[:] = tts
    nc.close()

tt=0
for t in out_ar.time:
    df_loc = ar_points.ivt.sel(time=t)
    ll = xr.plot.contour(df_loc,levels=[0,1])
    for item in ll.collections:
        for i in item.get_paths():
            v = i.vertices
            crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
            if (crit>=20): # AR has to be at least 20 deg lat 
                xx=(v[:, 0]/res_lon).astype(int)
                yy=((v[:, 1]+90)/res_lat).astype(int)
                for (x,y) in zip(xx,yy):
                    out_ar.ivt[y,tt,x]
    tt = tt+1
    plt.close()
        
out_ar.to_nectdf()
#from functions import to_nc
#to_nc(out_ar, lat_.values, lon_.values, ar_points.time , str(yy)+'_ivt_crit.nc')

<xarray.Dataset>
Dimensions:    (lat: 96, time: 365, lon: 144)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    quantile   float64 ...
    member_id  <U8 'r1i1p1f1'
  * time       (time) object 2014-01-01 12:00:00 ... 2014-12-31 12:00:00
Data variables:
    ivt        (lat, time, lon) int64 dask.array<chunksize=(96, 66, 144), meta=np.ndarray>


/srv/conda/envs/notebook/lib/python3.9/site-packages/xarray/plot/plot.py:1417: UserWarning: No contour levels were found within the data range.
  primitive = ax.contour(x, y, z, **kwargs)


AttributeError: 'Dataset' object has no attribute 'to_nectdf'

In [ ]:
AR = xr.open_dataset(str(yy)+'_crit.nc')
ivt_ = AR.ivt
ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon') # add extra points for AR at the edge of the world

In [ ]:
out_detect = np.zeros((ivt.shape[0],ivt.shape[1],ivt.shape[2]))

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1:
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10:
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc


for k in range(ivt.shape[0]):
    print(k)
    matrix = ivt[k,:,:]
    tst = floodfill(matrix)
    out_detect[k,:,:] = tst
    
# add the values of the extra points to get ARs at the end of the world
out_detect[:,:,:5] = out_detect[:,:,:5]+out_detect[:,:,144:]
out_detect = out_detect[:,:,:144]
out_detect[out_detect>1] = 1


lat_ = AR.lat
lon_ = AR.lon


def to_nc(indata, lat_test, lon_test, tts, fln):
    nc = Dataset(fln, 'w')
    lat_dim = nc.createDimension('lat', indata.shape[1])
    lon_dim = nc.createDimension('lon', indata.shape[2])
    t_dim = nc.createDimension('time', indata.shape[0])
    lat_var = nc.createVariable('lat', np.float64, ('lat'))
    lat_var[:] = lat_test
    lon_var = nc.createVariable('lon', np.float64, ('lon'))
    lon_var[:] = lon_test
    tnd = nc.createVariable('ivt', np.float64, ('time','lat','lon'))
    tnd[:,:,:] = indata
    times = nc.createVariable('time', np.float64, ('time'))
    times[:] = tts
    nc.close()

to_nc(out_ar, lat_.values, lon_.values, range(AR.ivt.shape[0]) , str(year)+'_AR_detection.nc')